In [2]:
from abc import ABC,abstractmethod

class NotificationSender(ABC):
    @abstractmethod
    def send(self,messege) ->str: ...

class EmailNotification(NotificationSender):
    def send(self,messege):
        return f"[email] sending: {messege}"
    
class SMSNotification(NotificationSender):
    def send(self,messege):
        return f"[sms] sending: {messege}"
    
class Notification(ABC):
    def __init__(self,sender) -> None:
        self.sender = sender

    def notify(self,messege:str): ...
        
class AlertNotification(Notification):
    def notify(self,messege:str):
        return self.sender.send(messege)

email = EmailNotification()
sms = SMSNotification()

alert = AlertNotification(email)
print(alert.notify("система перегружена"))

alert.sender = sms
print(alert.notify("низкий заряд батареи"))

[email] sending: система перегружена
[sms] sending: низкий заряд батареи


In [3]:
class PushNotification:
    def push(self,content:str):
        return f"[push] sending: {content}"

class PushNotificationAdapter(NotificationSender):
    def __init__(self,adaptee:PushNotification) -> None:
        self.adaptee = adaptee

    def send(self,messege) ->str:
        return self.adaptee.push(messege)
    

push = PushNotification()
adapter = PushNotificationAdapter(push)

alert = AlertNotification(adapter)
print(alert.notify("новый способ сообщения"))


[push] sending: новый способ сообщения


In [7]:
class BaseNotification(ABC):
    @abstractmethod
    def notify(self,messege) ->str: ...

class SimpleNotification(BaseNotification):
    def __init__(self,sender) ->None:
        self.sender = sender

    def notify(self,messege) -> str:
        return self.sender.send(messege)
    
class UrgentNotification(BaseNotification):
    def __init__(self,wrapped: BaseNotification) ->None:
        self.wrapped = wrapped

    def notify(self,messege) -> str:
        return "[urgent]"+""+ self.wrapped.notify(messege)
    
email = EmailNotification()
simple = SimpleNotification(email)

urgent = UrgentNotification(simple)
urgent.notify("сервер упал")


'[urgent][email] sending: сервер упал'

In [ ]:
class WaterHeater:
    def heat(self):
        print("нагрев воды")

class CoffeeGrinder:
    def grind(self):
        print("молем кофе")

class CoffeeBreaker:
    def brew(self):
        print("варим кофе")

class CoffeeMachine:
    def __init__(self) -> None:
        self.header = WaterHeater()
        self.grider = CoffeeGrinder()
        self.brewer = CoffeeBreaker()